In [1]:
#general purpose packages
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

#data processing
import re, string
#import emoji
import nltk

from sklearn import preprocessing
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split


#Naive Bayes
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

#transformers
!pip install transformers
from transformers import BertTokenizerFast
from transformers import TFBertModel
from transformers import RobertaTokenizerFast
from transformers import TFRobertaModel

#keras
import tensorflow as tf
from tensorflow import keras


#metrics
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix

#set seed for reproducibility
seed=42

#set style for plots
sns.set_style("whitegrid")
sns.despine()
plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True)
plt.rc("axes", labelweight="bold", labelsize="large", titleweight="bold", titlepad=10)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.5 MB/s eta 0:00:00


<ipython-input-1-335695ed4efd>:45: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use("seaborn-whitegrid")


<Figure size 640x480 with 0 Axes>

In [2]:
df=pd.read_csv("Bully_Dataset_Positive_Negative_Neutral.csv")

In [4]:
def remove_stopwords(sentence):
    """
    Removes a list of stopwords

    Args:
        sentence (string): sentence to remove the stopwords from

    Returns:
        sentence (string): lowercase sentence without the stopwords
    """
    # List of stopwords
    stopwords = ["a", "about", "above", "after", "again", "ago","against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]

    # Sentence converted to lowercase-only
    sentence = str(sentence).lower()

    words = sentence.split()
    no_words = [w for w in words if w not in stopwords]
    sentence = " ".join(no_words)

    return sentence

df['Clean_coment']=df['Clean_coment'].apply(lambda x:remove_stopwords(x))




In [5]:

import nltk
from nltk.stem import WordNetLemmatizer

# Download WordNet from NLTK
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')
# Initialize the WordNet Lemmatizer
lemmatizer = WordNetLemmatizer()

def lemmi(sentence):


    # Tokenize the sentence into individual words
    words = nltk.word_tokenize(sentence)

    # Lemmatize each word to its base form
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]

    # Join the lemmatized words back into a sentence
    lemmatized_sentence = " ".join(lemmatized_words)

    # Print the original and lemmatized sentences

    return  lemmatized_sentence
df['Clean_coment']=df['Clean_coment'].apply(lambda x:lemmi(x))
df["Clean_coment"]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


0                                              nice video
1                                   theme music nostalgic
2                                                    nice
3                                          won team super
4                                                     dar
                              ...                        
3577    middle order will get dozen stepping t let s h...
3578    just first match series team play positive can...
3579                                  new graphic amazing
3580                 cricket team please fix middle order
3581    bad performance batting can expect defend low ...
Name: Clean_coment, Length: 3582, dtype: object

In [6]:
MAX_LEN=128

In [7]:
def tokenize(data,max_len=MAX_LEN) :
    input_ids = []
    attention_masks = []
    for i in range(len(data)):
        encoded = tokenizer.encode_plus(
            data[i],
            add_special_tokens=True,
            max_length=MAX_LEN,
            padding='max_length',
            return_attention_mask=True
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    return np.array(input_ids),np.array(attention_masks)

In [8]:
def create_model(bert_model, max_len=MAX_LEN):

    ##params###
    opt = tf.keras.optimizers.Adam(learning_rate=1e-5, decay=1e-7)
    loss = tf.keras.losses.CategoricalCrossentropy()
    accuracy = tf.keras.metrics.CategoricalAccuracy()


    input_ids = tf.keras.Input(shape=(max_len,),dtype='int32')

    attention_masks = tf.keras.Input(shape=(max_len,),dtype='int32')

    embeddings = bert_model([input_ids,attention_masks])[1]

    output = tf.keras.layers.Dense(3, activation="softmax")(embeddings)

    model = tf.keras.models.Model(inputs = [input_ids,attention_masks], outputs = output)

    model.compile(opt, loss=loss, metrics=accuracy)


    return model

In [9]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [11]:
token_lens = []

for txt in df['Clean_coment'].values:
    tokens = tokenizer.encode(txt, max_length=512, truncation=True)
    token_lens.append(len(tokens))

max_len=np.max(token_lens)
print(f"MAX TOKENIZED SENTENCE LENGTH: {max_len}")

MAX TOKENIZED SENTENCE LENGTH: 64


In [12]:
token_lens = []

for i,txt in enumerate(df['Clean_coment'].values):
    tokens = tokenizer.encode(txt, max_length=512, truncation=True)
    token_lens.append(len(tokens))
    if len(tokens)>80:
        print(f"INDEX: {i}, TEXT: {txt}")


In [13]:
df['token_lens'] = token_lens

In [14]:
df = df.sort_values(by='token_lens', ascending=False)
df.head(20)

,ID,Output,Clean_coment,token_lens
1475,1517,neutral,old orthodox style cricket even useless ten no...,64
409,431,neutral,well well every single match till date except ...,55
2598,2681,positive,brilliant knock t cricket talk target no optio...,51
176,188,neutral,amir please humble n consistent not indulge me...,50
1496,1538,negative,can not win without good middle order pathetic...,48
3484,3581,neutral,make play no pure opener yes got position dome...,47
3131,3223,positive,bowling world cup injured play foreign always ...,47
2990,3082,neutral,main problem approach even today no shot groun...,45
888,918,neutral,world s best th s appreciate already yet missi...,43
3147,3239,positive,c grade performance c grade team team going t ...,43


In [15]:
df = df.sample(frac=1).reset_index(drop=True)

In [16]:
# df['Label'] = df['Label'].map({'Eartquake':0,'Flood':1,'Accident':2,'Other Disaster':3})

In [18]:
#In this step we are Encoding Our target variable which is Theme as we can see numbers from 0-6 have been assinged to every unique label
import numpy as np
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
Label=df['Output']
Label=np.array(Label).reshape(-1,1)


df['encoded_labels']=''
enc=le.fit_transform(Label)

df['encoded_labels']=enc
df.head()

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,ID,Output,Clean_coment,token_lens,encoded_labels
0,1347,neutral,need learn win match losing position,8,1
1,1617,positive,fantastic,3,2
2,906,positive,well best luck team next match thank p c b gre...,16,2
3,3611,negative,terrible please improve content,6,0
4,1680,neutral,rd class performance rd class team,8,1


In [19]:
ros = RandomOverSampler()
train_x, train_y = ros.fit_resample(np.array(df['Clean_coment']).reshape(-1, 1), np.array(df['Output']).reshape(-1, 1));
train_os = pd.DataFrame(list(zip([x[0] for x in train_x], train_y)), columns = ['Clean_coment', 'Output']);

In [20]:
X = train_os['Clean_coment'].values
y = train_os['Label'].values

In [21]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, stratify=y, random_state=seed)

In [22]:
new_index = pd.RangeIndex(len(X_train))
X_train = pd.Series(X_train, index=new_index)
train_input_ids, train_attention_masks = tokenize(X_train, MAX_LEN)
val_input_ids, val_attention_masks = tokenize(X_valid, MAX_LEN)
# test_input_ids, test_attention_masks = tokenize(X_test, MAX_LEN)

In [23]:
y_train_le = y_train.copy()
y_valid_le = y_valid.copy()

In [24]:
ohe = preprocessing.OneHotEncoder()
y_train = ohe.fit_transform(np.array(y_train).reshape(-1, 1)).toarray()
y_valid = ohe.fit_transform(np.array(y_valid).reshape(-1, 1)).toarray()

In [25]:
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [36]:
def create_model(bert_model, max_len=MAX_LEN):

    ##params###
    import tensorflow.keras.backend as K
    opt = tf.keras.optimizers.Adam(learning_rate=1e-5)
    loss = tf.keras.losses.CategoricalCrossentropy()
   # accuracy = tf.keras.metrics.CategoricalAccuracy()
    class F1Score(tf.keras.metrics.Metric):
      def __init__(self, name='f1_score', **kwargs):
        super().__init__(name=name, **kwargs)
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()

      def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

      def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()

      def result(self):
        p = self.precision.result()
        r = self.recall.result()
        return 2 * ((p * r) / (p + r + K.epsilon()))
    METRICS = [
      tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      F1Score(name='f1_score')]


    input_ids = tf.keras.Input(shape=(max_len,),dtype='int32')

    attention_masks = tf.keras.Input(shape=(max_len,),dtype='int32')

    embeddings = bert_model([input_ids,attention_masks])[1]

    output = tf.keras.layers.Dense(3, activation="softmax")(embeddings)

    model = tf.keras.models.Model(inputs = [input_ids,attention_masks], outputs = output)

    model.compile(opt, loss=loss, metrics=METRICS)


    return model

In [30]:
model = create_model(bert_model, MAX_LEN)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 128)]        0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 128)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  109482240   ['input_3[0][0]',                
                                thPoolingAndCrossAt               'input_4[0][0]']                
                                tentions(last_hidde                                               
                                n_state=(None, 128,                                         

In [31]:
history_bert = model.fit([train_input_ids,train_attention_masks], y_train, validation_data=([val_input_ids,val_attention_masks], y_valid), epochs=20, batch_size=5)

Epoch 1/20
1014/1014 [==============================] - ETA: 0s - loss: 0.5807 - accuracy: 0.7585 - precision: 0.8092 - recall: 0.6871 - f1_score: 0.7431

/usr/local/lib/python3.10/dist-packages/keras/engine/training.py:2448: UserWarning: Metric F1Score implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()


1014/1014 [==============================] - 250s 192ms/step - loss: 0.5807 - accuracy: 0.7585 - precision: 0.8092 - recall: 0.6871 - f1_score: 0.7431 - val_loss: 0.4722 - val_accuracy: 0.8162 - val_precision: 0.8262 - val_recall: 0.8060 - val_f1_score: 0.8160
Epoch 2/20
1014/1014 [==============================] - 178s 175ms/step - loss: 0.2842 - accuracy: 0.8968 - precision: 0.9011 - recall: 0.8915 - f1_score: 0.8963 - val_loss: 0.3221 - val_accuracy: 0.8667 - val_precision: 0.8689 - val_recall: 0.8628 - val_f1_score: 0.8658
Epoch 3/20
1014/1014 [==============================] - 179s 176ms/step - loss: 0.1704 - accuracy: 0.9394 - precision: 0.9403 - recall: 0.9386 - f1_score: 0.9395 - val_loss: 0.3816 - val_accuracy: 0.8612 - val_precision: 0.8652 - val_recall: 0.8604 - val_f1_score: 0.8628
Epoch 4/20
1014/1014 [==============================] - 176s 174ms/step - loss: 0.1160 - accuracy: 0.9599 - precision: 0.9609 - recall: 0.9599 - f1_score: 0.9604 - val_loss: 0.3368 - val_accuracy

In [35]:
from transformers import BertTokenizer
import tensorflow as tf

# Step 1: Load the tokenizer
tokenizerSaved = BertTokenizer.from_pretrained('bert-base-uncased')

# Step 2: Define the function to create the model
#def create_model(bert_model, max_len=128):
    # ... (rest of the model creation code)

# Step 3: Create the model
model = create_model(bert_model, max_len=128)

# Step 4: Load the weights directly to the model from memory
# Replace 'weights_list' with the actual list of weights for your model
#model.set_weights(weights_list)

# Function to preprocess the input text and make predictions
def predict_sentiment(input_text, model, tokenizer):
    MAX_LEN = 128

    # Step 5: Tokenize the input text
    inputs = tokenizer.encode_plus(input_text, add_special_tokens=True, max_length=MAX_LEN, return_tensors='tf')

    # Step 6: Make predictions using the loaded model
    predictions = model.predict([inputs['input_ids'], inputs['attention_mask']])

    # Step 7: Interpret the model's predictions
    predicted_class = tf.argmax(predictions, axis=1).numpy()[0]

    # Assuming you have a label mapping dictionary
    label_map = {0: 'good', 1: 'average', 2: 'bad'}
    predicted_label = label_map[predicted_class]

    return predicted_label

# Take input from the user
input_text = input("Enter your text: ")

# Make prediction
predicted_sentiment = predict_sentiment(input_text, model, tokenizerSaved)

print("Predicted Sentiment:", predicted_sentiment)


Enter your text: he is good player


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


ValueError: ignored

In [ ]:
# Step 1: Tokenize the input text using the tokenizer
input_text = 'SLEEPY JOE BIDEN IS PROPOSING THE BIGGEST TAX HIKE IN OUR COUNTRIES HISTORY! CAN ANYBODY REALLY VOTE FOR THIS?'
inputs = tokenizerSaved.encode_plus(input_text, add_special_tokens=True, return_tensors='tf')

# Step 2: Make predictions using the loaded model
prediction = new_model.predict([inputs['input_ids'], inputs['attention_mask']])

# Step 3: Interpret the model's predictions
predicted_class = np.argmax(prediction)

# Assuming you have a label mapping dictionary
label_map = {0: 'good', 1: 'average', 2: 'bad'}
predicted_label = label_map[predicted_class]

print("Input Text:", input_text)
print("Predicted Label:", predicted_label)
